# Simple End to End
## Generation and simulation of the fpgaconvnet IP

This part of the tutorial demonstrates the basic end-to-end flow from an ONNX model to a generated hardware IP. We encourage you to run this example to try our automated toolflow, and then follow the rest of the tutorial which provides a more detailed explanation of our codebase.

In [ ]:
import pathlib
from fpgaconvnet.parser.Parser import Parser
from fpgaconvnet.platform.Platform import Platform
from fpgaconvnet.optimiser.solvers import GreedyPartition

# load network
network_name = "single_layer"
onnx_path = f"{network_name}.onnx"
parser = Parser(custom_onnx=False, batch_size=1)
net = parser.onnx_to_fpgaconvnet(onnx_path)

device_path = "../../fpgaconvnet-optimiser/examples/platforms/zedboard.toml"
platform = Platform()
platform.update(device_path)
platform.port_width   = 64

# Design Space Exploration
opt = GreedyPartition(net, platform)
opt.bram_to_lut = False
opt.off_chip_streaming = False
opt.balance_bram_uram = False
opt.rsc_allocation = 0.75
opt.transforms = []
opt.transforms_probs = []

opt.run_solver()
opt.update_partitions()

# export configuration and prediction report
pathlib.Path(network_name).mkdir(parents=True, exist_ok=True)
opt.create_report(f"{network_name}/report.json")
opt.net.save_all_partitions(f"{network_name}/config.json")

throughput = -opt.get_cost()
print(f"predicted throughput (img/s): {throughput}")
print(f"predicted resource usage: {net.partitions[0].get_resource_usage()}")

In [ ]:
import toml
from fpgaconvnet.hls.generate.network import GenerateNetwork

with open(device_path, "r") as f:
    device_name = toml.load(f)["device"]["part"]

# generate HLS code
gen_net = GenerateNetwork(network_name, f"{network_name}/config.json", f"{network_name}.onnx", device_name)
gen_net.create_partition_project(0)

In [ ]:
import numpy as np

input_image = np.random.rand(1,1,28,28).astype(np.float32)
# run C simulation
gen_net.run_testbench(0, input_image)

In [ ]:
# run C synthesis and export IP
gen_net.generate_partition_hardware(0)

In [ ]:
gen_net.run_cosimulation(0, input_image)

## Troubleshooting 
1. In `Module.py` and `Module3D.py` from `/fpgaconvnet-model/fpgaconvnet/models/modules/`, comment out the line:
   ```python
   # from xgboost import XGBRegressor
   ```
2. If the version of numpy cannot compile, you may need to uninstall numpy and reinstall a downgraded version. Version 2.0.0 is recommended. 
3. You may be working on a higher version of Linux OS that doesn't support Vivado 2019.1. In this case, Vivado may fail to simulate. Try deleting the ld folder at: `/Xilinx/Vivado/2022.2/tps/lnx64/binutils-2.26/bin/ld`

## What's next
Now that an ip has been generated, you may proceed to `hardware-tutorial.ipynb` to integrate the IP to a project. 